In [4]:
import os
from lava.utils.system import Loihi2
loihi2_is_available = Loihi2.is_loihi2_available
if loihi2_is_available:
    # Enable SLURM, the workload manager used to distribute Loihi2 resources to users
    os.environ['SLURM'] = '1'
    os.environ['PARTITION'] = "kp"
    os.environ['LOIHI_GEN'] = "N3B3"
import numpy as np
from lava.proc.sdn.process import SigmaDelta, Sigma
from lava.proc.io.source import RingBuffer as SourceBuffer
from lava.proc.io.sink import RingBuffer as SinkBuffer
from lava.proc.embedded_io.spike import PyToNxAdapter, NxToPyAdapter
import matplotlib.pyplot as plt
from src.lava.lib.dl.slayer.state_space_models.s4 import S4D
import torch
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi2HwCfg, Loihi2SimCfg
from lava.proc.dense.process import Dense
from lava.proc.sdn.process import ActivationMode
from lava.proc.s4d.process import SigmaS4dDelta, S4d
from lava.proc.sparse.process import Sparse
from lava.proc.spike_integrator.process import SpikeIntegrator
import random
from lava.magma.compiler.subcompilers.nc.ncproc_compiler import CompilerOptions

# S4d
random.seed(10)
np.random.seed(10)
torch.manual_seed(10)

In [5]:
# S4 stuff
model_dim = 256
d_states = 16

model = S4D(d_model=model_dim,
            d_state=d_states,
            dropout=False,
            transposed=True,
            quantize = False,
            lr=None,
            skip=False,
            activation=None,
            is_real=True)
model = model.to('cpu')
model.eval()

S4D(
  (layer): FFTConv(
    (activation): Identity()
    (kernel): SSMKernelDiag()
    (drop): Identity()
    (drop_kernel): Identity()
  )
  (mult_activation): Identity()
  (drop): Identity()
  (output_linear): Identity()
)

In [6]:

n_steps = 1000 # sequence length
inp_exp = 13
inp = (torch.rand(1, model_dim, n_steps) * 2** inp_exp).int().float()


model.setup_step()
A = model.layer.kernel.dA.detach().numpy()
B = model.layer.kernel.dB.detach().numpy()
C = model.layer.kernel.dC.detach().numpy()


kron_matrix = np.kron(np.eye(model_dim), np.ones((d_states, )))

#encoder
# S4D
dense_2 = Sparse(weights=kron_matrix.T, weight_exp=6, num_message_bits=24)
neuron = S4d(shape=((model_dim * d_states,)), state_exp = 6, s4_exp = 12,    # state exp doesn't get used here, currently hard coded to 6
                        a=A.flatten().reshape((model_dim * d_states,)),
                        b=B.flatten().reshape((model_dim * d_states,)),
                        c=C.flatten().reshape((model_dim * d_states,)))
dense_3 = Sparse(weights=kron_matrix, num_message_bits=24, weight_exp=6)
relay = SpikeIntegrator(shape=(model_dim,), relu_flag=False, q_scale=1/(2**6), q_exp=6)


weights =np.random.rand(model_dim, model_dim)
weights[weights < 0.9] = 0
weights *= 10

# FF
dense_4 = Dense(weights=weights.astype(int), num_message_bits=24)
ff_neuron = SpikeIntegrator(shape=(model_dim,), relu_flag=True)



# connect
dense_2.a_out.connect(neuron.a_in)  
neuron.s_out.connect(dense_3.s_in)
dense_3.a_out.connect(relay.a_in)
relay.s_out.connect(dense_4.s_in)
dense_4.a_out.connect(ff_neuron.a_in)


run_cfg = Loihi2HwCfg()


dense_2.run(
                condition=RunSteps(num_steps=n_steps), run_cfg=run_cfg
                )
dense_2.stop()





end


Partitioning converged after iteration=6
Per core utilization:
-------------------------------------------------------------------------
| AxonIn |NeuronGr| Neurons|Synapses| AxonMap| AxonMem|  Total |  Cores |
|-----------------------------------------------------------------------|
|   1.60%|  12.50%|   3.10%|  51.20%|   0.79%|   0.00%|  43.52%|       3|
|  25.60%|  12.50%|   6.25%|  25.60%|   1.60%|   0.00%|  43.52%|       1|
|   1.60%|  12.50%|  99.98%|   3.21%|  12.79%|   0.00%|  34.55%|       3|
|-----------------------------------------------------------------------|
| Total                                                        |       7|
-------------------------------------------------------------------------
